In [210]:
# coding:utf-8
# Textual analysis through the open web
from __future__ import print_function
import io
import os
import bs4
import sys
import json
import time
import urllib
import random
import requests
import wikipedia

In [227]:
def read_json(path):
    data = ''
    with io.open(path, 'r', encoding='utf-8') as f:
        data = json.loads(f.read())
        print(__message(u'Loaded {}'.format(path)))
    return data
    
def modify_game_meta(data):
    for k, v in data.items():
        data[k].pop('Last', None)
        data[k]['Year'] = int(data[k]['First'][:4])
        data[k].pop('First', None)
        data[k].pop('Title', None)
        data[k]['Summary'] = scrape_wiki(k)
        
#         # Scraping Duck Duck Go
#         print(__message('Scraping DuckDuckGo for a list of URLs...'))
#         new_links = scrape_duckduckgo(k, data[k]['Developer'])
#         data[k]['Links'] = new_links | set(data[k]['Links'])

    print(__message('Done'))
    return data

def scrape_wiki(title):
    try:
        searchstring = title
        summary = wikipedia.summary(searchstring)
        print(__success(searchstring))
    except wikipedia.DisambiguationError:
        try:
            searchstring = u'{} (video game)'.format(title).replace(u' ', u'_')
            summary = wikipedia.page(searchstring, auto_suggest=False).summary
            print(__success(searchstring))
        except wikipedia.DisambiguationError:
            searchstring = u'{} (Unix video game)'.format(title).replace(u' ', u'_')
            summary = wikipedia.page(searchstring, auto_suggest=False).summary
            print(__success(searchstring))
        except wikipedia.PageError:
            summary = 'No summary found on Wikipedia.'
            print(__warning(u'Wikipedia cannot find "{}"'.format(searchstring)))
    except wikipedia.PageError:
        try:
            summary = wikipedia.page(searchstring, auto_suggest=False).summary
            print(__success(searchstring))
        except wikipedia.PageError:
            summary = 'No summary found on Wikipedia.'
            print(__warning(u'Search term "{}" returned nothing'.format(searchstring)))
    return summary

def scrape_duckduckgo(keywords, developer=""):
    searchstring = u'"{}" {} {}'.format(keywords, developer, u'interview game')
    response = requests.get(u'http://duckduckgo.com/html/?q={}'.format(
                urllib.quote(searchstring.encode('utf-8'))),
                            timeout=(9.1, 12.1)
             )
    print(__message(u'DDG: {}'.format(searchstring)))
    soup = bs4.BeautifulSoup(response.text)
    links = []
    for node in soup.select('div.web-result'):
        if 'web-result-sponsored' in node['class']:
            continue
        try:
            links.append(node.select('a.large')[0].get('href'))
        except Exception as e:
            print(__failure(e))
            pass
    if links:
        print(__success(u'DDG: {}'.format(searchstring)))
        print('\t\t\n'.join(links))
    return links

def scrape_url(url):
    try:
        response = requests.get(url, timeout=(10, 15))
        return response.text
    except Exception as e:
        print(__failure(e))

def clean_url_data(links, data):
    links = set(links)
    for link in data.keys():
        if link not in links:
            data.pop(link, None)
    return data
    
def save_json(path, data):
    with io.open(path, 'w', encoding='utf-8') as f:
        try:
            output = json.dumps(data, indent=2, ensure_ascii=False)
            f.write(output)
        except UnicodeEncodeError:
            f.write(output.encode('utf-8'))
    print(__message(u'Written to {}'.format(path)))
        
def __success(text):
    return u'  (SUCC) {}'.format(text).encode('utf-8')
    
def __failure(text):
    return u'!!FAIL!! {}'.format(text).encode('utf-8')
    
def __warning(text):
    return u'??WARN?? {}'.format(text).encode('utf-8')
    
def __message(text):
    return u'   |MSG| {}'.format(text).encode('utf-8')

In [231]:
cwd = os.getcwd()

game_meta = read_json(os.path.join(cwd, 'data', 'game-sources.json'))

shuffled_game_meta = game_meta.items()
random.shuffle(shuffled_game_meta)
for game, meta in shuffled_game_meta:
#     game_meta[game]['Links'] += scrape_duckduckgo(game, game_meta[game]['Developer'])
#     game_meta[game]['Links'] = list(set(game_meta[game]['Links']))
    save_json(os.path.join(cwd, 'data', 'game-sources.json'), game_meta)
    time.sleep(1)

   |MSG| Loaded /Users/spaxe/dev/roguelike-universe/data/game-sources.json
   |MSG| DDG: "Z.H.P. Unlosing Ranger VS Darkdeath Evilman" Nippon Ichi Software interview game
  (SUCC) DDG: "Z.H.P. Unlosing Ranger VS Darkdeath Evilman" Nippon Ichi Software interview game
http://www.gamespot.com/z-h-p-unlosing-ranger-vs-darkdeath-evilman/videos/		
http://www.gamespot.com/z-h-p-unlosing-ranger-vs-darkdeath-evilman/		
http://www.gamefaqs.com/psp/975347-zhp-unlosing-ranger-vs-darkdeath-evilman/critic		
http://www.siliconera.com/2013/04/18/nippon-ichi-software-ceo-sohei-niikawa-reflects-upon-past-mistakes/		
https://en.wikipedia.org/wiki/Nippon_Ichi_Software		
http://www.tech-gaming.com/the-guided-fate-paradox/		
http://ncloud.blogspot.com/2013/07/japans-favourite-nippon-ichi-games.html		
https://www.gizorama.com/2013/console/playstation-3/the-guided-fate-paradox-review		
http://www.vg247.com/2011/08/10/nippon-ichi-keen-on-psp-remasters/		
http://ncloud12.blogspot.com/2013/07/japan-favourite-nip